In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import LongformerModel, LongformerTokenizer, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm import tqdm

In [11]:
df = pd.read_csv('/kaggle/input/for-model-1/merged_df (1).csv')
df = df[['text', 'multi_labels','hier_label']]
# df['multi_labels'] = df['multi_labels'].apply(ast.literal_eval)

In [12]:
def list_ebal(df):
    # Преобразуем строки меток в списки (если они ещё не списки)
    df['multi_labels'] = df['multi_labels'].apply(eval)  # eval используется для преобразования строки в список
    # df['hier_label'] = df['hier_label'].apply(eval)  # аналогично для иерархических меток
    return df

df = list_ebal(df)

In [13]:
df.head(
)

,text,multi_labels,hier_label
0,"Трамп вновь пригрозил последствиями Ирану, есл...","[Международные отношения, Политика, Происшествия]","['Международные отношения', 'Конфликты']"
1,Ким Чен Ын пообещал скоро представить новое ст...,"[Международные отношения, Политика, Вооружение]","['Международные отношения', 'Политика']"
2,"В России почти 6,5 миллиона человек встретили ...","[Общество, Региональные новости, Политика]","['Общество', 'Праздники']"
3,Столкновения произошли в Гонконге во время сог...,"[Международные отношения, Происшествия, Политика]","['Международные отношения', 'Протесты']"
4,"Папа Римский Франциск извинился за то, что нак...","[Международные отношения, Религия, Общество]","['Международные отношения', 'Религия']"


In [14]:
# ================================
# 1. Определяем модель
# ================================

class HierarchicalMultiTaskLongformer(nn.Module):
    def __init__(self, model_name, num_multi_labels, num_hier_labels):
        """
        :param model_name: имя предобученной модели, например, "allenai/longformer-base-4096"
        :param num_multi_labels: число классов для мультилейблинга
        :param num_hier_labels: число классов для иерархической классификации
        """
        super(HierarchicalMultiTaskLongformer, self).__init__()
        self.longformer = LongformerModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.1)
        # Голова для мультилейблинга
        self.multi_label_classifier = nn.Linear(self.longformer.config.hidden_size, num_multi_labels)
        # Голова для иерархической классификации
        self.hierarchical_classifier = nn.Linear(self.longformer.config.hidden_size, num_hier_labels)
        # Функции потерь
        self.multi_label_loss_fn = nn.BCEWithLogitsLoss()
        self.hierarchical_loss_fn = nn.CrossEntropyLoss()
    
    def forward(self, input_ids, attention_mask, multi_labels=None, hier_labels=None):
        outputs = self.longformer(input_ids, attention_mask=attention_mask)
        # Используем представление первого токена ([CLS])
        cls_output = outputs.last_hidden_state[:, 0, :]
        cls_output = self.dropout(cls_output)
        ml_logits = self.multi_label_classifier(cls_output)
        hier_logits = self.hierarchical_classifier(cls_output)
        
        loss = None
        if multi_labels is not None and hier_labels is not None:
            ml_loss = self.multi_label_loss_fn(ml_logits, multi_labels)
            hier_loss = self.hierarchical_loss_fn(hier_logits, hier_labels)
            loss = ml_loss + hier_loss  # Можно добавить веса для балансировки
        return ml_logits, hier_logits, loss


In [15]:
# ================================
# 2. Определяем Dataset для новостных данных
# ================================

class NewsDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length, mlb, hier_label_map):
        """
        :param dataframe: DataFrame с колонками 'text', 'multi_labels', 'hier_label'
        :param tokenizer: токенизатор модели
        :param max_length: максимальная длина последовательности
        :param mlb: объект MultiLabelBinarizer для мультилейблинга
        :param hier_label_map: словарь для маппинга иерархических меток в числа
        """
        self.df = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.mlb = mlb
        self.hier_label_map = hier_label_map
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        row = self.df.iloc[index]
        text = row['text']
        # Токенизация текста
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        input_ids = encoding['input_ids'].squeeze()          # [max_length]
        attention_mask = encoding['attention_mask'].squeeze()  # [max_length]
        
        # Преобразуем мультиметочные метки в бинарный вектор
        multi_labels = self.mlb.transform([row['multi_labels']])[0]
        multi_labels = torch.tensor(multi_labels, dtype=torch.float)
        
        # Преобразуем иерархическую метку в числовое значение
        hier_label_str = row['hier_label']
        hier_label = self.hier_label_map[hier_label_str]
        hier_label = torch.tensor(hier_label, dtype=torch.long)
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'multi_labels': multi_labels,
            'hier_labels': hier_label
        }

In [16]:
# ================================
# 3. Загрузка данных и подготовка разметки
# ================================
def create_hier_label_map(df):
    """Создаём маппинг для иерархических меток."""
    unique_hier_labels = df['hier_label'].explode().unique()  # Разворачиваем списки и находим уникальные метки
    hier_label_map = {label: idx for idx, label in enumerate(unique_hier_labels)}
    return hier_label_map

In [17]:
# ================================
# 4. Тренировочный цикл
# ================================

def train_model(model, dataloader, optimizer, scheduler, device, epochs):
    model.train()
    for epoch in tqdm(range(epochs)):
        epoch_loss = 0.0
        for batch in dataloader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            multi_labels = batch['multi_labels'].to(device)
            hier_labels = batch['hier_labels'].to(device)
            
            ml_logits, hier_logits, loss = model(input_ids, attention_mask, multi_labels, hier_labels)
            loss.backward()
            optimizer.step()
            scheduler.step()
            epoch_loss += loss.item()
        avg_loss = epoch_loss / len(dataloader)
        print(f"Epoch {epoch+1}/{epochs}, Average Loss: {avg_loss:.4f}")
    return model

In [18]:
# Параметры обучения
model_name = "allenai/longformer-base-4096"
max_length = 512
batch_size = 4
epochs = 10
learning_rate = 2e-5

# Создаём маппинг для иерархических меток
hier_label_map = create_hier_label_map(df)
num_hier_labels = len(hier_label_map)

# Определяем фиксированный набор классов для мультилейблинга (знаем заранее)
# Получаем уникальные элементы
unique_elements = list(set([item for sublist in df['multi_labels'] for item in sublist]))
all_multi_labels = unique_elements
mlb = MultiLabelBinarizer(classes=all_multi_labels)
mlb.fit(df['multi_labels'])
num_multi_labels = len(all_multi_labels)

# Инициализируем токенизатор и модель
tokenizer = LongformerTokenizer.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HierarchicalMultiTaskLongformer(model_name, num_multi_labels, num_hier_labels)
model.to(device)

# Создаём Dataset и DataLoader
dataset = NewsDataset(df, tokenizer, max_length, mlb, hier_label_map)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [19]:
# Оптимизатор и scheduler
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
total_steps = len(dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Обучение модели
trained_model = train_model(model, dataloader, optimizer, scheduler, device, epochs)

# Сохраняем веса модели
torch.save(trained_model.state_dict(), "fine_tuned_longformer.pt")
print("Модель сохранена как fine_tuned_longformer.pt")

 10%|█         | 1/10 [00:13<02:02, 13.57s/it]

Epoch 1/10, Average Loss: 3.6785


 20%|██        | 2/10 [00:26<01:43, 12.97s/it]

Epoch 2/10, Average Loss: 3.2754


 30%|███       | 3/10 [00:38<01:29, 12.77s/it]

...

 90%|█████████ | 9/10 [01:53<00:12, 12.55s/it]

Epoch 9/10, Average Loss: 2.4398


100%|██████████| 10/10 [02:06<00:00, 12.63s/it]

Epoch 10/10, Average Loss: 2.3252


Модель сохранена как fine_tuned_longformer.pt


In [21]:
import torch
from transformers import LongformerTokenizer

# Параметры модели (эти параметры должны соответствовать тем, что использовались при обучении)
model_name = "allenai/longformer-base-4096"
num_multi_labels = num_multi_labels   # например, если у нас 5 меток для мультилейблинга
num_hier_labels = num_hier_labels   # например, если у нас 3 класса для иерархической классификации

# Инициализация модели и загрузка сохранённых весов
model = HierarchicalMultiTaskLongformer(model_name, num_multi_labels, num_hier_labels)
model.load_state_dict(torch.load("fine_tuned_longformer.pt", map_location=torch.device("cpu"), weights_only=False))
model.eval()  # Переводим модель в режим инференса

# Инициализация токенизатора
tokenizer = LongformerTokenizer.from_pretrained(model_name)

# Пример входного текста
sample_text = "Новая экономическая инициатива обсуждается на высшем уровне."

# Токенизация текста
inputs = tokenizer(
    sample_text,
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512
)

# Выполнение инференса (без вычисления градиентов)
with torch.no_grad():
    ml_logits, hier_logits, _ = model(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"]
    )

# Преобразование логитов в вероятности
ml_probs = torch.sigmoid(ml_logits)       # Вероятности для мультилейблинга
hier_probs = torch.softmax(hier_logits, dim=1)  # Вероятности для иерархической классификации

print("Логиты для мультилейблинга:", ml_logits)
print("Вероятности для мультилейблинга:", ml_probs)
print("Логиты для иерархической классификации:", hier_logits)
print("Вероятности для иерархической классификации:", hier_probs)

# Дополнительно: если требуется получить CLS-токен (вектор эмбеддингов)
# Его размерность будет [batch_size, hidden_size] (например, [1, 768] для base модели)
# Обычно CLS-токен используется внутри модели для классификации, но можно его извлечь так:
with torch.no_grad():
    outputs = model.longformer(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"]
    )
cls_token_output = outputs.last_hidden_state[:, 0, :]  # Вектор CLS токена
print("Вектор CLS токена:", cls_token_output)


Логиты для мультилейблинга: tensor([[-1.9676, -1.6637, -0.8333,  0.7046, -1.5735, -2.0407, -2.4252, -1.4855,
         -1.5834, -1.2678, -1.7714, -0.8604, -0.4174, -2.2043, -2.3345, -1.4021,
         -1.9903,  0.9811, -1.6408, -0.7815]])
Вероятности для мультилейблинга: tensor([[0.1226, 0.1593, 0.3029, 0.6692, 0.1717, 0.1150, 0.0813, 0.1846, 0.1703,
         0.2196, 0.1454, 0.2973, 0.3971, 0.0994, 0.0883, 0.1975, 0.1202, 0.7273,
         0.1624, 0.3140]])
Логиты для иерархической классификации: tensor([[ 0.6467,  1.1650, -1.0914, -0.5558, -0.6637, -0.1290,  0.3603,  1.2253,
         -0.7078, -0.2891,  0.0159, -0.9071,  1.2728,  1.9451,  1.1688, -0.5179,
         -0.1692,  0.3464, -0.6740,  0.0645, -1.2849, -0.4379, -0.2997, -1.4251,
         -0.3742, -0.9130, -1.1035, -0.7495, -0.0920,  0.0205,  0.2829, -0.2161]])
Вероятности для иерархической классификации: tensor([[0.0466, 0.0782, 0.0082, 0.0140, 0.0126, 0.0214, 0.0350, 0.0831, 0.0120,
         0.0183, 0.0248, 0.0098, 0.0871, 0.1706, 

In [22]:
# Пример входного текста
sample_text = "Пираты напали на судно  и похитили трех членов экипажа, после чего в перестрелке убили четырех сотрудников спасательной группы ВМС страны, сообщают местные СМИ."
# Токенизация текста
inputs = tokenizer(
    sample_text,
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=max_length
)
# Переносим тензоры на устройство
inputs = {k: v.to('cpu') for k, v in inputs.items()}

# Выполняем инференс
with torch.no_grad():
    ml_logits, hier_logits, _ = model(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"]
    )

# Обработка результатов для мультилейблинга
hier_label_names = [label for label, idx in sorted(hier_label_map.items(), key=lambda x: x[1])]
ml_probs = torch.sigmoid(ml_logits)[0]  # [num_multi_labels]
threshold = 0.5
predicted_multi_labels = [all_multi_labels[i] for i, prob in enumerate(ml_probs) if prob > threshold]

# Обработка результатов для иерархической классификации
hier_probs = torch.softmax(hier_logits, dim=1)[0]  # [num_hier_labels]
predicted_hier_index = torch.argmax(hier_probs).item()
predicted_hier_label = hier_label_names[predicted_hier_index]

# Извлекаем вектор CLS токена
with torch.no_grad():
    outputs = model.longformer(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"]
    )
cls_token_vector = outputs.last_hidden_state[:, 0, :]  # размер: [batch_size, hidden_size]

# Вывод результатов
print("Предсказанные метки для мультилейблинга:")
print(predicted_multi_labels)
print("\nВероятности для мультилейблинга:")
for label, prob in zip(all_multi_labels, ml_probs.tolist()):
    print(f"{label}: {prob:.4f}")

print("\nПредсказанная метка для иерархической классификации:")
print(predicted_hier_label)
print("\nВероятности для иерархической классификации:")
for label, prob in zip(hier_label_names, hier_probs.tolist()):
    print(f"{label}: {prob:.4f}")

print("\nВектор CLS токена:")
print(cls_token_vector)

Предсказанные метки для мультилейблинга:
['Политика', 'Международные отношения']

Вероятности для мультилейблинга:
Законодательство: 0.1395
Экономика: 0.1979
Безопасность: 0.3065
Политика: 0.5960
Преступность: 0.1385
Кино: 0.1566
Энергетика: 0.2101
Культура: 0.1471
Терроризм: 0.2068
Спорт: 0.2589
Туризм: 0.1493
Происшествия: 0.3129
Региональные новости: 0.1946
Вооружение: 0.1942
Экология: 0.2052
Религия: 0.2335
Здравоохранение: 0.1278
Международные отношения: 0.7009
Транспорт: 0.1326
Общество: 0.2634

Предсказанная метка для иерархической классификации:
['Международные отношения', 'Политика']

Вероятности для иерархической классификации:
['Международные отношения', 'Конфликты']: 0.0196
['Международные отношения', 'Политика']: 0.2212
['Общество', 'Праздники']: 0.0304
['Международные отношения', 'Протесты']: 0.0129
['Международные отношения', 'Религия']: 0.0172
['Транспорт', 'Ж/Д']: 0.0114
['Экономика', 'Бизнес']: 0.0181
['Политика', 'Внутренняя политика']: 0.0631
['Спорт', 'Баскетбол']: